In [1]:
!pip install -q unsloth flask ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 13.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 98.1 M

In [2]:
from flask import Flask, request, jsonify
from unsloth import FastLanguageModel
from transformers import pipeline
from threading import Thread
import os
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-06 20:56:47.913013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754513808.137752      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754513808.204149      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/gemma-hr-lora/gemma_hr_lora",  # or your dataset path
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
    device_map = "auto"
)

# Pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Flask app
app = Flask(__name__)

==((====))==  Unsloth 2025.8.1: Fast Gemma patching. Transformers: 4.55.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.8.1 patched 18 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
Device set to use cuda:0


In [4]:
@app.route("/infer", methods=["POST"])
def infer():
    data = request.json
    resume = data.get("resume", "")
    jd = data.get("jd", "")
    task = data.get("task", "match")

    if task == "match":
        prompt = f"""
You are a private AI hiring assistant.

Your task is to compare the following resume with the job description and give a match score out of 100, followed by a one-line explanation.

[RESUME]
{resume}

[JOB DESCRIPTION]
{jd}

[OUTPUT]
Match Score:
"""

    elif task == "qa":
        prompt = f"""
You are an AI assistant helping an interviewer prepare for a technical interview.

Given the resume and job description, generate 5 relevant technical interview questions tailored to the candidate's profile.

[RESUME]
{resume}

[JOB DESCRIPTION]
{jd}

[OUTPUT]
Interview Questions:
"""

    else:
        return jsonify({"result": "Invalid task."})

    output = pipe(prompt.strip(), max_new_tokens=300)[0]["generated_text"]

    # Only return the generated response (remove the prompt)
    if "[OUTPUT]" in prompt:
        response = output.split("[OUTPUT]")[-1].strip()
    else:
        response = output.strip()

    return jsonify({"result": response})


In [5]:
# Run Flask in a background thread
def run():
    app.run(host='0.0.0.0', port=5000)

Thread(target=run).start()

 * Serving Flask app '__main__'
 * Debug mode: off


In [6]:
!pip install pyngrok
!ngrok config add-authtoken 30mAxdtHJ1reyNU5dLTqSP3EWsL_32j53pvMCD5QoAG4uQFyg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [7]:
from pyngrok import ngrok 

public_url = ngrok.connect(addr="http://localhost:5000")
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://3bfcabddf447.ngrok-free.app" -> "http://localhost:5000"
